In [1]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
import time

preprocessed_data = pd.read_csv('processed_emotions_dataset.csv',index_col=0)
preprocessed_data['preprocessed_text_split'] = preprocessed_data['preprocessed_text'].str.split()
preprocessed_data = preprocessed_data.dropna()
print(preprocessed_data.head(10))

                                                text     label  \
0      i just feel really helpless and heavy hearted      fear   
1  ive enjoyed being able to slouch about relax a...   sadness   
2  i gave up my internship with the dmrg and am f...      fear   
3                         i dont know i feel so lost   sadness   
4  i am a kindergarten teacher and i am thoroughl...      fear   
5         i was beginning to feel quite disheartened   sadness   
6  i would think that whomever would be lucky eno...      love   
7  i fear that they won t ever feel that deliciou...       joy   
8  im forever taking some time out to have a lie ...  surprise   
9  i can still lose the weight without feeling de...   sadness   

                                   preprocessed_text  \
0          just feel realli helpless and heavi heart   
1  ive enjoy be abl to slouch about relax and unw...   
2  gave up my internship with the dmrg and am fee...   
3                             dont know feel so l

In [2]:
# Function to retrieve top few number of each category
def get_top_data(preprocessed_data, top_n):
    top_data_joy = preprocessed_data[preprocessed_data['label'] == 'joy'].head(top_n)
    top_data_sadness = preprocessed_data[preprocessed_data['label'] == 'sadness'].head(top_n)
    top_data_anger = preprocessed_data[preprocessed_data['label'] == 'anger'].head(top_n)
    top_data_fear = preprocessed_data[preprocessed_data['label'] == 'fear'].head(top_n)
    top_data_love = preprocessed_data[preprocessed_data['label'] == 'love'].head(top_n)
    top_data_surprise = preprocessed_data[preprocessed_data['label'] == 'surprise'].head(top_n)
    data_equal_size_per_label = pd.concat([top_data_joy, top_data_sadness, top_data_anger, top_data_fear, top_data_love, top_data_surprise])
    return data_equal_size_per_label

# Function call to get the top 15000 from each sentiment
data_equal_size_per_label = get_top_data(preprocessed_data, top_n=15000)

# After selecting top few samples of each sentiment
print("After segregating and taking equal number of rows for each sentiment:")
print(data_equal_size_per_label['label'].value_counts())
data_equal_size_per_label.head(10)

After segregating and taking equal number of rows for each sentiment:
label
joy         15000
sadness     15000
anger       15000
fear        15000
love        15000
surprise    15000
Name: count, dtype: int64


,text,label,preprocessed_text,preprocessed_text_split
7,i fear that they won t ever feel that deliciou...,joy,fear that they won ever feel that delici excit...,"[fear, that, they, won, ever, feel, that, deli..."
10,i try to be nice though so if you get a bitchy...,joy,tri to be nice though so if you get bitchi per...,"[tri, to, be, nice, though, so, if, you, get, ..."
12,i have officially graduated im not feeling as ...,joy,have offici graduat im not feel a ecstat a tho...,"[have, offici, graduat, im, not, feel, a, ecst..."
14,i feel my portfolio demonstrates how eager i a...,joy,feel my portfolio demonstr how eager am to lea...,"[feel, my, portfolio, demonstr, how, eager, am..."
15,i may be more biased than the next because i h...,joy,may be more bias than the next becaus have dep...,"[may, be, more, bias, than, the, next, becaus,..."
16,i didn t feel terrific,joy,didn feel terrif,"[didn, feel, terrif]"
21,i am feeling much stronger and more confident ...,joy,am feel much stronger and more confid now and ...,"[am, feel, much, stronger, and, more, confid, ..."
22,i take a shower i feel wonderful energetic and...,joy,take shower feel wonder energet and all my pre...,"[take, shower, feel, wonder, energet, and, all..."
26,i feel like i am actually getting something us...,joy,feel like am actual get someth use out of it,"[feel, like, am, actual, get, someth, use, out..."
27,i was able to overcome this anxiousness and fe...,joy,wa abl to overcom thi anxious and feel peac a ...,"[wa, abl, to, overcom, thi, anxious, and, feel..."


In [3]:
# Skip-gram model (sg = 1)
size = 1000
window = 3
min_count = 1
workers = 3
sg = 1

word2vec_model_file = 'word2vec_' + str(size) + '.model'
start_time = time.time()
stemmed_tokens = pd.Series(data_equal_size_per_label['preprocessed_text_split']).values
# Train the Word2Vec Model
w2v_model = Word2Vec(stemmed_tokens, min_count = min_count, vector_size = size, workers = workers, window = window, sg = sg)
print("Time taken to train word2vec model: " + str(time.time() - start_time))
w2v_model.save(word2vec_model_file)

sg_w2v_model = Word2Vec.load(word2vec_model_file)


Time taken to train word2vec model: 19.15617799758911


In [5]:
# Train Test Split Function
def split_train_test(data_equal_size_per_label, test_size=0.3, shuffle_state=True):
    X_train, X_test, Y_train, Y_test = train_test_split(data_equal_size_per_label[['text','preprocessed_text_split']], 
                                                        data_equal_size_per_label['label'], 
                                                        shuffle=shuffle_state,
                                                        test_size=test_size, 
                                                        random_state=15)
    print("Value counts for Train sentiments")
    print(Y_train.value_counts())
    print("Value counts for Test sentiments")
    print(Y_test.value_counts())
    # X_train = X_train.reset_index()
    # X_test = X_test.reset_index()
    # Y_train = Y_train.to_frame()
    # Y_train = Y_train.reset_index()
    # Y_test = Y_test.to_frame()
    # Y_test = Y_test.reset_index()
    return X_train, X_test, Y_train, Y_test

X_train, X_test, Y_train, Y_test = split_train_test(data_equal_size_per_label)


Value counts for Train sentiments
label
fear        10572
love        10546
sadness     10545
joy         10508
anger       10427
surprise    10402
Name: count, dtype: int64
Value counts for Test sentiments
label
surprise    4598
anger       4573
joy         4492
sadness     4455
love        4454
fear        4428
Name: count, dtype: int64


In [6]:
# Store the vectors for train data in following file
word2vec_filename = 'train_word2vec.csv'
with open(word2vec_filename, 'w') as word2vec_file:
    for index, row in X_train.iterrows():
        model_vector = (np.mean([sg_w2v_model.wv[token] for token in row['preprocessed_text_split']], axis=0)).tolist() # this is where we can add the probabliities from LDA to do a weighted avg instead
        # if index == 0:
        #     header = ",".join(str(ele) for ele in range(1000))
        #     word2vec_file.write(header)
        #     word2vec_file.write("\n")
        # Check if the line exists else it is vector of zeros
        if type(model_vector) is list:  
            line1 = ",".join( [str(vector_element) for vector_element in model_vector] )
        else:
            line1 = ",".join([str(0) for i in range(1000)])
        word2vec_file.write(line1)
        word2vec_file.write('\n')

In [16]:
# Load from the filename
word2vec_df = pd.read_csv(word2vec_filename, header=None)
#Initialize the model
clf_decision_word2vec = DecisionTreeClassifier()

start_time = time.time()
# Fit the model
clf_decision_word2vec.fit(word2vec_df, Y_train)
print("Time taken to fit the model with word2vec vectors: " + str(time.time() - start_time))

# Test model
test_features_word2vec = []
for index, row in X_test.iterrows():
    model_vector = np.mean([sg_w2v_model.wv[token] for token in row['preprocessed_text_split']], axis=0).tolist() # this is where we can add the probabliities from LDA to do a weighted avg instead
    if type(model_vector) is list:
        test_features_word2vec.append(model_vector)
    else:
        test_features_word2vec.append(np.array([0 for i in range(1000)]))
test_predictions_word2vec = clf_decision_word2vec.predict(test_features_word2vec)
print(classification_report(Y_test,test_predictions_word2vec))

Time taken to fit the model with word2vec vectors: 99.00531816482544
              precision    recall  f1-score   support

       anger       0.24      0.24      0.24      4573
        fear       0.22      0.24      0.23      4428
         joy       0.25      0.25      0.25      4492
        love       0.29      0.29      0.29      4454
     sadness       0.25      0.24      0.25      4455
    surprise       0.32      0.30      0.31      4598

    accuracy                           0.26     27000
   macro avg       0.26      0.26      0.26     27000
weighted avg       0.26      0.26      0.26     27000

